<a href="https://colab.research.google.com/github/miacatharina/mimic-code/blob/main/Final_estimation_estimator_selection_potassium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install econml[dowhy]

### Setup

In [ ]:
import pandas as pd
import os, warnings, random
import warnings
import econml 
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from econml.dml import DML, LinearDML, SparseLinearDML, CausalForestDML
import numpy as np
from itertools import product
from sklearn.linear_model import (Lasso, LassoCV, LogisticRegression,
                                  LogisticRegressionCV,LinearRegression,
                                  MultiTaskElasticNet,MultiTaskElasticNetCV)
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from econml.score import RScorer
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner
from econml.dml import CausalForestDML, LinearDML, NonParamDML
from econml.dr import DRLearner, ForestDRLearner, LinearDRLearner
from joblib import Parallel, delayed
from econml.metalearners import TLearner, SLearner, XLearner
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.base import clone
import logging
import sys
from econml.dml import DML, LinearDML,SparseLinearDML,CausalForestDML, NonParamDML, KernelDML
from matplotlib import pyplot as plt
import copy
from econml.sklearn_extensions.model_selection import GridSearchCVList
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import dowhy
from dowhy import CausalModel
from sklearn.linear_model import LassoCV, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from econml.sklearn_extensions.linear_model import WeightedLassoCV, WeightedLasso
from econml.cate_interpreter import SingleTreeCateInterpreter
import shap
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import tensorflow as tf
import timeit
from sklearn.model_selection import train_test_split
from econml.cate_interpreter import SingleTreePolicyInterpreter


def seed_everything(seed=123):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    tf.random.set_seed(seed)

seed = 123
seed_everything(seed)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
#Check the version of TensorFlow you are using
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.9264132790000303
GPU (s):
0.10025079299998652
GPU speedup over CPU: 29x
Found GPU at: /device:GPU:0
2.11.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


### Import the data

In [ ]:
XTY = pd.read_csv("/content/XTY.csv")
X = pd.read_csv("/content/X.csv")
Y = pd.read_csv("/content/Y.csv")
T = pd.read_csv("/content/T.csv")
XTY_train = pd.read_csv("/content/XTY_train.csv")
XTY_test = pd.read_csv("/content/XTY_test.csv")
X_train = pd.read_csv("/content/X_train.csv")
X_test = pd.read_csv("/content/X_test.csv")
Y_train = pd.read_csv("/content/Y_train.csv")
Y_test = pd.read_csv("/content/Y_test.csv")
T_train = pd.read_csv("/content/T_train.csv")
T_test = pd.read_csv("/content/T_test.csv")

XTY.set_index("stay_id", inplace=True)
X.set_index("stay_id", inplace=True)
Y.set_index("stay_id", inplace=True)
T.set_index("stay_id", inplace=True)
XTY_train.set_index("stay_id", inplace=True)
XTY_test.set_index("stay_id", inplace=True)
X_train.set_index("stay_id", inplace=True)
X_test.set_index("stay_id", inplace=True)
Y_train.set_index("stay_id", inplace=True)
Y_test.set_index("stay_id", inplace=True)
T_train.set_index("stay_id", inplace=True)
T_test.set_index("stay_id", inplace=True)

T_train = T_train.iloc[:, 0].to_numpy()
T_test = T_test.iloc[:, 0].to_numpy()
X1_train = X_train.iloc[:, 3:4]
X1_test = X_test.iloc[:, 3:4]
XT = pd.concat([X, pd.DataFrame(T, columns=['treated'])], axis=1)
XT_train = pd.concat([X_train, pd.DataFrame(T_train, columns=['treated'])], axis=1)
XT_test = pd.concat([X_test, pd.DataFrame(T_test, columns=['treated'])], axis=1)


#(1) Binary outcome: Hypokalemia within 48 hours 
Y1A = Y.iloc[:, 2].to_numpy()
Y1A_train = Y_train.iloc[:, 2].to_numpy()
Y1A_test = Y_test.iloc[:, 2].to_numpy()

#(1)Continuous outcome: Minimum serum potassium within 48 hours 
Y1B = Y.iloc[:, 0]#.to_numpy()
Y1B_train = Y_train.iloc[:, 0].to_numpy()
Y1B_test = Y_test.iloc[:, 0].to_numpy()

In [ ]:
print(Y1B)

stay_id
33267162    3.8
30655167    4.5
30002548    3.9
30029209    4.3
30031157    3.8
           ... 
39917784    3.3
39926643    3.0
39933856    3.9
39966638    3.8
39989105    6.9
Name: 0_48_min_k, Length: 5444, dtype: float64


# Set up Grid Search Baseline

In [ ]:
class GridSearchCVList:

    def __init__(self, estimator_list, param_grid_list, scoring=None,
                 n_jobs=None, iid='warn', refit=True, cv='warn', verbose=0, pre_dispatch='2*n_jobs',
                 error_score='raise', return_train_score=False):
        self._gcv_list = [GridSearchCV(estimator, param_grid, scoring=scoring,
                                       n_jobs=n_jobs, refit=refit, cv=cv, verbose=verbose,
                                       pre_dispatch=pre_dispatch, error_score=error_score,
                                       return_train_score=return_train_score)
                          for estimator, param_grid in zip(estimator_list, param_grid_list)]
        return

    def fit(self, X, y, sample_weight = None, **fit_params):
        self.best_ind_ = np.argmax([gcv.fit(X, y, sample_weight = sample_weight, **fit_params).best_score_ for gcv in self._gcv_list])
        self.best_estimator_ = self._gcv_list[self.best_ind_].best_estimator_
        self.best_score_ = self._gcv_list[self.best_ind_].best_score_
        self.best_params_ = self._gcv_list[self.best_ind_].best_params_
        return self

    def predict(self, X):
        return self.best_estimator_.predict(X)

    def predict_proba(self, X):
        return self.best_estimator_.predict_proba(X)
  
#Cross-validated nuisance and final stage models
def first_stage_reg():
    return GridSearchCVList([Lasso(random_state=123),
                             RandomForestRegressor(n_estimators=100, random_state=123),
                             GradientBoostingRegressor(random_state=123)],
                             param_grid_list=[{'alpha': [.001, .01, .1, 1, 10]},
                                               {'max_depth': [3, None],
                                               'min_samples_leaf': [10, 50]},
                                              {'n_estimators': [50, 100],
                                               'max_depth': [3],
                                               'min_samples_leaf': [10, 30]}],
                             cv=5,
                             scoring='neg_mean_squared_error')

def first_stage_clf():
    return GridSearchCVList([LogisticRegression(random_state=123),
                             RandomForestClassifier(n_estimators=100, random_state=123),
                             GradientBoostingClassifier(random_state=123)],
                             param_grid_list=[{'C': [0.01, .1, 1, 10, 100]},
                                              {'max_depth': [3, 5],
                                               'min_samples_leaf': [10, 50]},
                                              {'n_estimators': [50, 100],
                                               'max_depth': [3],
                                               'min_samples_leaf': [10, 30]}],
                             cv=5,
                             scoring='neg_mean_squared_error')

def final_stage():
    return GridSearchCVList([WeightedLasso(random_state=123),
                             RandomForestRegressor(n_estimators=100, random_state=123)],
                             param_grid_list=[{'alpha': [.001, .01, .1, 1, 10]},
                                              {'max_depth': [3, 5],
                                               'min_samples_leaf': [10, 50]}],
                             cv=5,
                             scoring='neg_mean_squared_error')

## Best nuisance models: Binary outcome 


In [ ]:
model_y = clone(first_stage_clf().fit(X, Y1A).best_estimator_)
print(model_y)

LogisticRegression(C=100, random_state=123)


In [ ]:
model_t = clone(first_stage_clf().fit(X, T).best_estimator_)
print(model_t)

GradientBoostingClassifier(min_samples_leaf=10, random_state=123)


In [ ]:
model_yt = clone(first_stage_clf().fit(XT, Y1A).best_estimator_)
print(model_yt)

LogisticRegression(C=100, random_state=123)


In [ ]:
#Binary outcomes: Best estimators for propensity model and outcome model
model_ya = LogisticRegression(C=100, random_state=123, n_jobs=1)
model_ta = GradientBoostingClassifier(min_samples_leaf=10, random_state=123)
model_yta = LogisticRegression(C=100, random_state=123, n_jobs=1)

## Best nuisance models: Continuous outcome 

In [ ]:
model_yb = clone(first_stage_reg().fit(X, Y1B).best_estimator_)
print(model_yb)

GradientBoostingRegressor(min_samples_leaf=30, n_estimators=50,
                          random_state=123)


In [ ]:
model_ytb = clone(first_stage_reg().fit(XT, Y1B).best_estimator_)

In [ ]:
print(model_ytb)

GradientBoostingRegressor(min_samples_leaf=30, n_estimators=50,
                          random_state=123)


In [ ]:
#Continous outcomes: Best estimators for propensity model and outcome model
model_yb = GradientBoostingRegressor(min_samples_leaf=30, n_estimators=50,
                          random_state=123)
model_tb = GradientBoostingClassifier(min_samples_leaf=10, random_state=123)
model_ytb =GradientBoostingRegressor(min_samples_leaf=30, n_estimators=50,
                          random_state=123)

# Set up the RScorer

## Fit the RScorer: Binary outcome 

In [ ]:
# Train the RScore on the data 
#Follow the same procedure for selecting the nuisance models for metrics as for CATE estimators 
scorer_a = RScorer(model_y=first_stage_clf(), model_t=first_stage_clf(),
                 discrete_treatment=True, random_state=123)

In [ ]:
scorer_a.fit(Y1A_test, T_test, X=X_test)

## Fit the RScorer: Continuous outcome 

In [ ]:
# Train the RScore on the data 
#Follow the same procedure for selecting the nuisance models for metrics as for CATE estimators 
scorer_b = RScorer(model_y=first_stage_reg(), model_t=first_stage_clf(),
                 discrete_treatment=True, random_state=123)

In [ ]:
scorer_b.fit(Y1B_test, T_test, X=X_test)

# Set up influence function 

In [ ]:
X_train = pd.read_csv("/content/X_train.csv") 
X_test = pd.read_csv("/content/X_test.csv")
Y_train = pd.read_csv("/content/Y_train.csv")
Y_test = pd.read_csv("/content/Y_test.csv")
T_train = pd.read_csv("/content/T_train.csv")
T_test = pd.read_csv("/content/T_test.csv")
XTY_train = pd.read_csv("/content/XTY_train.csv")


XTY_train.set_index("stay_id", inplace=True)
X_train.set_index("stay_id", inplace=True)
X_test.set_index("stay_id", inplace=True)
T_train.set_index("stay_id", inplace=True)
T_test.set_index("stay_id", inplace=True)

#T_train = T_train.iloc[:, 0].to_numpy()
XT_train = pd.concat([X_train, pd.DataFrame(T_train, columns=['treated'])], axis=1)
XT_test = pd.concat([X_test, pd.DataFrame(T_test, columns=['treated'])], axis=1)

In [ ]:
xgb_plugin1 = LogisticRegression(C=10, random_state=123)
xgb_plugin0 = LogisticRegression(C=1, random_state=123)

In [ ]:
x0 = XT_train.loc[XT_train['treated'] == 0]
y0 = XTY_train.loc[XTY_train['treated'] == 0, '0_48_hypokalemia']
xgb_plugin0.fit(x0, y0)

LogisticRegression(C=1, random_state=123)

In [ ]:
x1 = XT_train.loc[XT_train['treated'] != 0]
y1 = XTY_train.loc[XTY_train['treated'] != 0, '0_48_hypokalemia']
xgb_plugin1.fit(x1, y1)

LogisticRegression(C=10, random_state=123)

In [ ]:
y_pred0 = xgb_plugin0.predict(XT_test)
y_pred1 = xgb_plugin1.predict(XT_test)

In [ ]:
#Estimated treatment effect using the plug-in method (difference between the predicted outcomes of two XGBoost models trained separately on the control group and the treatment group)
t_plugin = y_pred1 - y_pred0

In [ ]:
#Compute the propensity score using the gradient boosting classifier model and the treatment variable
gb = GradientBoostingClassifier(min_samples_leaf=10, n_estimators=50,
                           random_state=123)
gb.fit(X_train, T_train)
ps = gb.predict_proba(X_test)[:, 1]
print(ps)

[0.82123387 0.11883833 0.64019843 ... 0.97021595 0.2471682  0.21186484]


In [ ]:
#Use the PS to compute a weight matrix for the treatment effect estimate
T_test = T_test.iloc[:, 0].to_numpy()
a = (T_test - ps)
ident = np.array([1] * len(ps))
c = (ps * (ident - ps))
#weight vector used to adjust the treatment effect estimate based on the probability of receiving the treatment (propensity score)
b = np.array([2] * len(T_test)) * T_test * (T_test - ps) / c

# (A) Estimator selection: Binary outcome


## 1. LinearDML

### 1.1 Select best performing nuisance model class and hyperparameters

In [ ]:
ldml = LinearDML(model_y=model_ya,
                model_t=model_ta,
                random_state=123)

In [ ]:
ldml.fit(Y1A_train, T_train, X=X_train)

In [ ]:
ldml_cate = ldml.effect(X_test)
ldml_cate_inference = ldml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(ldml_cate_inference)

               Uncertainty of Mean Point Estimate               
mean_point stderr_mean zstat  pvalue ci_mean_lower ci_mean_upper
----------------------------------------------------------------
    -0.013       0.018 -0.689  0.491        -0.043         0.018
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.188          -0.285            0.25
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.189         -0.374          0.352
------------------------------------------


### 1.2 Calculate the robustness

In [ ]:
ldml.fit(Y=Y1A_test, T=T_test, X=X_test)
ldml_cate_test = ldml.effect(X_test)

ERMSE = ((ldml_cate-ldml_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.3595974464239379


### 1.3 Calculate the R-Score 

In [ ]:
rscore = scorer_a.score(ldml)
print(rscore)

-0.027509915024941378


## 2. LinearDRLearner 

### 2.1 Select best performing nuisance model class and hyperparameters

In [ ]:
LinearDR_learner = LinearDRLearner(model_regression=model_yta, model_propensity=model_ta, random_state=123)

In [ ]:
LinearDR_learner.fit(Y1A_train, T_train, X=X_train)

In [ ]:
LinearDR_learner_cate = LinearDR_learner.effect(X_test)

In [ ]:
LinearDR_learner_cate_inference = LinearDR_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(LinearDR_learner_cate_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
    -0.001       0.016 -0.08  0.936        -0.027         0.025
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.138          -0.211           0.205
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.139         -0.299          0.293
------------------------------------------


### 2.2 Calculate the robustness 

In [ ]:
LinearDR_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
LinearDR_learner_cate_test = LinearDR_learner.effect(X_test)

ERMSE = ((LinearDR_learner_cate-LinearDR_learner_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.3793354485695427


### 3.3 Calculate the RScore 

In [ ]:
rscore = scorer_a.score(LinearDR_learner)
print(rscore)

-0.13707450374857366


## 3. S-Learner

### 3.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the SLearner
S_learner = SLearner(overall_model= first_stage_clf())

In [ ]:
# Train S_learner
best_estimator_s_learner = S_learner.fit(Y1A_train, T_train, X=X_train).overall_model.best_estimator_
print(best_estimator_s_learner)

LogisticRegression(C=10, random_state=123)


In [ ]:
# Estimate treatment effects on test data
S_learner = SLearner(overall_model=best_estimator_s_learner)
S_learner.fit(Y1A_train, T_train, X=X_train, inference='bootstrap')
S_te = S_learner.effect(X_test)

In [ ]:
S_te_inference = S_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(S_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
       0.0       0.074   0.0    1.0        -0.122         0.122
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
      0.0             0.0             0.0
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.074            nan            nan
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 3.2 Calculate the robustness

In [ ]:
S_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
S_te_test = S_learner.effect(X_test)

ERMSE = ((S_te-S_te_test)**2).mean()**(1/2)
print(ERMSE)

0.04285495643554833


### 3.3 Calculate the R-Score 

In [ ]:
rscore = scorer_a.score(S_learner)
print(rscore)

-0.002246796565891218


In [ ]:
#computing the mean squared error (square of the difference between two estimates of the treatment effect: the plug-in estimate and the X-learner estimate)
plug_in = (t_plugin-S_te)**2
#%%
#computing the loss/ error function for the estimated treatment effect
l_de = (ident - b) * t_plugin**2 + b*Y1_test*(t_plugin - S_te) + (- a*(t_plugin - S_te)**2 + S_te**2)
#%%
print((np.sum(l_de) + np.sum(plug_in)))

-47.56298636240794


## 4. T-learner

### 4.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the T_Learner
T_learner = TLearner(models=first_stage_clf())

In [ ]:
# Train T_learner
T_learner.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Assuming that T_learner.models is a list of the fitted GridSearchCVList objects
best_estimators = [model.best_estimator_ for model in T_learner.models]
print(best_estimators)

[GradientBoostingClassifier(min_samples_leaf=30, n_estimators=50,
                           random_state=123), GradientBoostingClassifier(min_samples_leaf=30, random_state=123)]


In [ ]:
best_estimators = [GradientBoostingClassifier(min_samples_leaf=30, n_estimators=50,
                           random_state=123), GradientBoostingClassifier(min_samples_leaf=30, random_state=123)]

In [ ]:
# Estimate treatment effects on test data
T_learner = TLearner(models=best_estimators)
T_learner.fit(Y1A_train, T_train, X=X_train, inference='bootstrap')
T_te = T_learner.effect(X_test)

In [ ]:
T_te_inference = T_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(T_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.022       0.121 0.179  0.858        -0.178         0.221
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.128          -0.176           0.235
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.176         -0.255          0.314
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 4.2 Calculate the robustness

In [ ]:
T_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
T_te_test = T_learner.effect(X_test)

ERMSE = ((T_te-T_te_test)**2).mean()**(1/2)
print(ERMSE)

0.3880681356019908


### 4.3 Calculate the R-Score 

In [ ]:
rscore = scorer_a.score(T_learner)
print(rscore)

0.1871661450246357


## 5. X-learner

### 5.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the X_Learner
X_learner = XLearner(models=first_stage_clf(), propensity_model=first_stage_clf())

In [ ]:
# Train X_learner
X_learner.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Assuming that X_learner.models is a list of the fitted GridSearchCVList objects
best_estimators_models = [model.best_estimator_ for model in X_learner.models]

# best_estimators[0] contains the best LogisticRegression estimator
# best_estimators[1] contains the best RandomForestClassifier estimator
# best_estimators[2] contains the best GradientBoostingClassifier estimator
print(best_estimators_models)

[GradientBoostingClassifier(min_samples_leaf=30, n_estimators=50,
                           random_state=123), GradientBoostingClassifier(min_samples_leaf=30, random_state=123)]


In [ ]:
best_estimators_models = [GradientBoostingClassifier(min_samples_leaf=30, n_estimators=50,
                           random_state=123), GradientBoostingClassifier(min_samples_leaf=30, random_state=123)]

In [ ]:
best_estimator_pm = X_learner.propensity_models[0].best_estimator_
print(best_estimator_pm)

GradientBoostingClassifier(min_samples_leaf=30, random_state=123)


In [ ]:
best_estimator_pm = GradientBoostingClassifier(min_samples_leaf=30, random_state=123)

In [ ]:
# Estimate treatment effects on test data
X_learner = XLearner(models=best_estimators_models, propensity_model=best_estimator_pm)
X_learner.fit(Y1A_train, T_train, X=X_train, inference='bootstrap')
X_te = X_learner.effect(X_test)
X_te_inference = X_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(X_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.015       0.154 0.097  0.922        -0.238         0.268
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.135             0.0             0.0
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.204            nan            nan
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 5.2 Calculate the robustness

In [ ]:
X_learner.fit(Y=Y1A_test, T=T_test, X=X_test)
X_te_test = X_learner.effect(X_test)

ERMSE = ((X_te-X_te_test)**2).mean()**(1/2)
print(ERMSE)

0.21483630614728785


### 5.3 Calculate the R-Score

In [ ]:
rscore = scorer_a.score(X_learner)
print(rscore)

0.09131423951435569


## 6. Causal Forest DML 

### 6.1 Select best performing nuisance model and hyperparameter

In [ ]:
forestdml = CausalForestDML(model_y=model_ya, model_t=model_ta, discrete_treatment=True,random_state=seed)

In [ ]:
forestdml.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Estimate treatment effects on test data
forestdml_te = forestdml.effect(X_test)
forestdml_te_inference = forestdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdml_te_inference)

               Uncertainty of Mean Point Estimate               
mean_point stderr_mean zstat  pvalue ci_mean_lower ci_mean_upper
----------------------------------------------------------------
    -0.001       0.091 -0.008  0.993        -0.151         0.149
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.041          -0.064            0.07
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
         0.1         -0.155          0.164
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 6.2 Calculate the robustness

In [ ]:
forestdml.fit(Y=Y1A_test, T=T_test, X=X_test)
forestdml_te_test = forestdml.effect(X_test)

ERMSE = ((forestdml_te-forestdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.09384345792279752


### 6.3 Calculate the RScore

In [ ]:
rscore = scorer_a.score(forestdml)
print(rscore)

0.036048670577151


## 7. ForestDRLearner 

### 7.1 Select best performing nuisance model and hyperparameter

In [ ]:
#cv = KFold(n_splits=3, shuffle=True, random_state=123)
forestdrl = ForestDRLearner(model_regression=model_yta, model_propensity=model_ta, random_state=seed)
forestdrl.fit(Y1A_train, T_train, X=X_train, cache_values=True)
# Estimate treatment effects on test data
forestdrl_te = forestdrl.effect(X_test)
forestdrl_te_inference = forestdrl.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdrl_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.019       0.046 0.405  0.686        -0.057         0.094
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
     0.02          -0.014           0.054
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
        0.05         -0.062          0.102
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 7.2 Calculate the robustness

In [ ]:
forestdrl.fit(Y=Y1A_test, T=T_test, X=X_test)
forestdrl_te_test = forestdrl.effect(X_test)

ERMSE = ((forestdrl_te-forestdrl_te_test)**2).mean()**(1/2)
print(ERMSE)

0.047924235471375656


### 7.3 Calculate the RScore

In [ ]:
rscore = scorer_a.score(forestdrl)
print(rscore)

0.01542487041382945


## 8. NonParamDML 

### 8.1 Select best performing nuisance model and hyperparameter

In [ ]:
nonparamdml = NonParamDML(model_y=model_ya, model_t=model_ta, model_final=final_stage(),random_state=123)

In [ ]:
nonparamdml.fit(Y1A_train, T_train, X=X_train)

In [ ]:
# Access the best estimator for model_final after fitting the NonParamDML
best_estimator_model_final = nonparamdml.model_final_.best_estimator_
print(best_estimator_model_final)

RandomForestRegressor(max_depth=5, min_samples_leaf=10, random_state=123)


In [ ]:
best_estimator_model_final = RandomForestRegressor(max_depth=5, min_samples_leaf=10, random_state=123, n_jobs=1)

In [ ]:
nonparamdml = NonParamDML(model_y=model_yb, model_t=model_tb, model_final=best_estimator_model_final,random_state=123)
nonparamdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
nonparamdml_te = nonparamdml.effect(X_test)
nonparamdml_te_inference = nonparamdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(nonparamdml_te_inference)

Uncertainty of Mean Point Estimate
mean_point
----------
     0.053
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.058           -0.02           0.158
-----------------------------------------


### 8.2 Calculate the robustness

In [ ]:
nonparamdml.fit(Y=Y1A_test, T=T_test, X=X_test)
nonparamdml_te_test = nonparamdml.effect(X_test)

ERMSE = ((nonparamdml_te-nonparamdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.13718927198281988


### 8.3 Calculate the RScore

In [ ]:
rscore = scorer_a.score(nonparamdml)
print(rscore)

0.008302116152131611


# (B) Estimator selection: Continuous outcome


## 1. Linear DML 

### 1.1 Select the best performing nuisance model class and hyperparameters 

In [ ]:
ldml = LinearDML(model_y=model_yb,
                model_t=model_tb,
                random_state=123)

In [ ]:
ldml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
ldml_cate = ldml.effect(X_test)
ldml_cate_inference = ldml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(ldml_cate_inference)

               Uncertainty of Mean Point Estimate               
mean_point stderr_mean zstat  pvalue ci_mean_lower ci_mean_upper
----------------------------------------------------------------
    -0.026       0.018 -1.428  0.153        -0.057         0.004
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
     0.16          -0.251           0.196
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.161         -0.356          0.299
------------------------------------------


### 1.2 Calculate the robustness

In [ ]:
ldml.fit(Y=Y1B_test, T=T_test, X=X_test)
ldml_cate_test = ldml.effect(X_test)

ERMSE = ((ldml_cate-ldml_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.3771937007164605


### 1.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(ldml)
print(rscore)

0.029743847273251744


## 2. LinearDRLearner 

### 2.1 Select best performing nuisance model class and hyperparameters

In [ ]:
LinearDR_learner = LinearDRLearner(model_regression=model_ytb, model_propensity=model_tb, random_state=123)

In [ ]:
LinearDR_learner.fit(Y1B_train, T_train, X=X_train)

In [ ]:
LinearDR_learner_cate = LinearDR_learner.effect(X_test)

In [ ]:
LinearDR_learner_cate_inference = LinearDR_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(LinearDR_learner_cate_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.023       0.019 1.214  0.225        -0.008         0.054
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.146          -0.202           0.267
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.147         -0.307          0.379
------------------------------------------


 ### 2.2 Calculate the robustness

In [ ]:
LinearDR_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
LinearDR_learner_cate_test = LinearDR_learner.effect(X_test)

ERMSE = ((LinearDR_learner_cate-LinearDR_learner_cate_test)**2).mean()**(1/2)
print(ERMSE)

0.44331252765324997


### 2.3 Calculate the RScore


In [ ]:
rscore = scorer_b.score(LinearDR_learner)
print(rscore)

-0.00507321797271687


## 3. S-Learner 

### 3.1 Select best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the SLearner
S_learner = SLearner(overall_model= first_stage_reg())

In [ ]:
# Train S_learner
best_estimator_s_learner = S_learner.fit(Y1B_train, T_train, X=X_train).overall_model.best_estimator_
print(best_estimator_s_learner)

RandomForestRegressor(min_samples_leaf=10, random_state=123)


In [ ]:
best_estimator_s_learner = RandomForestRegressor(min_samples_leaf=10, random_state=123, n_jobs=1)

In [ ]:
# Estimate treatment effects on test data
S_learner = SLearner(overall_model=best_estimator_s_learner)
S_learner.fit(Y1B_train, T_train, X=X_train, inference='bootstrap')
S_te = S_learner.effect(X_test)

In [ ]:
S_te_inference = S_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(S_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.001        0.01 0.123  0.902        -0.015         0.017
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.007           -0.01           0.013
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.012         -0.017          0.021
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 3.2 Calculate the robustness

In [ ]:
S_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
S_te_test = S_learner.effect(X_test)

ERMSE = ((S_te-S_te_test)**2).mean()**(1/2)
print(ERMSE)

0.010073471562534431


### 3.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(S_learner)
print(rscore)

-0.00507321797271687


## 4. T-Learner 

### 4.1 Select the best performing nuisance model class and hyperparameters

In [ ]:
# Instantiate the T_Learner
T_learner = TLearner(models=first_stage_reg())

In [ ]:
# Train T_learner
T_learner.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Assuming that T_learner.models is a list of the fitted GridSearchCVList objects
best_estimators = [model.best_estimator_ for model in T_learner.models]
print(best_estimators)

[RandomForestRegressor(min_samples_leaf=10, random_state=123), RandomForestRegressor(min_samples_leaf=10, random_state=123)]


In [ ]:
best_estimators = [RandomForestRegressor(min_samples_leaf=10, random_state=123, n_jobs=1), RandomForestRegressor(min_samples_leaf=10, random_state=123, n_jobs=1)]

In [ ]:
# Estimate treatment effects on test data
T_learner = TLearner(models=best_estimators)
T_learner.fit(Y1B_train, T_train, X=X_train, inference='bootstrap')
T_te = T_learner.effect(X_test)

In [ ]:
T_te_inference = T_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(T_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.029       0.126 0.231  0.817        -0.178         0.236
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
     0.13          -0.192           0.243
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.181         -0.261          0.324
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 4.2 Calculate the robustness 

In [ ]:
T_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
T_te_test = T_learner.effect(X_test)

ERMSE = ((T_te-T_te_test)**2).mean()**(1/2)
print(ERMSE)

0.21301635522606827


### 4.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(T_learner)
print(rscore)

0.13251803843958443


## 5. X-learner 


### 5.1 Select the best performing nuisance model class and hyperparameters 

In [ ]:
# Instantiate the X_Learner
X_learner = XLearner(models=first_stage_reg(), propensity_model=first_stage_clf())

In [ ]:
# Train X_learner
X_learner.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Assuming that X_learner.models is a list of the fitted GridSearchCVList objects
best_estimators_models = [model.best_estimator_ for model in X_learner.models]

# best_estimators[0] contains the best LogisticRegression estimator
# best_estimators[1] contains the best RandomForestClassifier estimator
# best_estimators[2] contains the best GradientBoostingClassifier estimator
print(best_estimators_models)

[RandomForestRegressor(min_samples_leaf=10, random_state=123), RandomForestRegressor(min_samples_leaf=10, random_state=123)]


In [ ]:
best_estimators_models = [RandomForestRegressor(min_samples_leaf=10, random_state=123, n_jobs=1), RandomForestRegressor(min_samples_leaf=10, random_state=123, n_jobs=1)]


In [ ]:
best_estimator_pm = X_learner.propensity_models[0].best_estimator_
print(best_estimator_pm)

GradientBoostingClassifier(min_samples_leaf=30, random_state=123)


In [ ]:
best_estimator_pm = GradientBoostingClassifier(min_samples_leaf=30, random_state=123)


In [ ]:
# Estimate treatment effects on test data
X_learner = XLearner(models=best_estimators_models, propensity_model=best_estimator_pm)
X_learner.fit(Y1B_train, T_train, X=X_train, inference='bootstrap')
X_te = X_learner.effect(X_test)
X_te_inference = X_learner.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(X_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.026       0.079 0.333  0.739        -0.104         0.157
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.079          -0.084           0.156
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.112         -0.134          0.203
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 5.2 Calculate the robustness 

In [ ]:
X_learner.fit(Y=Y1B_test, T=T_test, X=X_test)
X_te_test = X_learner.effect(X_test)

ERMSE = ((X_te-X_te_test)**2).mean()**(1/2)
print(ERMSE)

0.1158891462801172


### 5.3 Calculate the RScore 

In [ ]:
rscore = scorer_b.score(X_learner)
print(rscore)

0.05586523221334394


## 6. NonParamDML 

### 6.1 Select the best performing nuisance model class and hyperparameters 

In [ ]:
nonparamdml = NonParamDML(model_y=model_yb, model_t=model_tb, model_final=final_stage(),random_state=123)

In [ ]:
nonparamdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Access the best estimator for model_final after fitting the NonParamDML
best_estimator_model_final = nonparamdml.model_final_.best_estimator_
print(best_estimator_model_final)

WeightedLasso(alpha=0.1, random_state=123)


In [ ]:
best_estimator_model_final = WeightedLasso(alpha=0.1, random_state=123)

In [ ]:
nonparamdml = NonParamDML(model_y=model_yb, model_t=model_tb, model_final=best_estimator_model_final,random_state=123)
nonparamdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
nonparamdml_te = nonparamdml.effect(X_test)
nonparamdml_te_inference = nonparamdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(nonparamdml_te_inference)

Uncertainty of Mean Point Estimate
mean_point
----------
    -0.017
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
      0.0          -0.017          -0.017
-----------------------------------------


### 6.1 Calculate the robustness

In [ ]:
nonparamdml.fit(Y=Y1B_test, T=T_test, X=X_test)
nonparamdml_te_test = nonparamdml.effect(X_test)

ERMSE = ((nonparamdml_te-nonparamdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.004225700500897298


### 6.3  Calculate the RScore

In [ ]:
rscore = scorer_b.score(nonparamdml)
print(rscore)

-0.0001055289898637124


## 7. CausalForestDML 

### 7.1 Select best performing nuisance model and hyperparameter

In [ ]:
forestdml = CausalForestDML(model_y=model_yb, model_t=model_tb, discrete_treatment=True, random_state=seed)

In [ ]:
forestdml.fit(Y1B_train, T_train, X=X_train)

In [ ]:
# Estimate treatment effects on test data
forestdml_te = forestdml.effect(X_test)
forestdml_te_inference = forestdml.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdml_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.024       0.118 0.203  0.839         -0.17         0.217
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.047          -0.051           0.107
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.127         -0.179           0.23
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 6.2 Calculate the robustness

In [ ]:
forestdml.fit(Y=Y1B_test, T=T_test, X=X_test)
forestdml_te_test = forestdml.effect(X_test)

ERMSE = ((forestdml_te-forestdml_te_test)**2).mean()**(1/2)
print(ERMSE)

0.0967172077704181


### 6.3 Calculate the RScore

In [ ]:
rscore = scorer_b.score(forestdml)
print(rscore)

0.05062296222877427


## 8. ForestDRLearner 


### 7.1 Select best performing nuisance model and hyperparameter

In [ ]:
forestdrl = ForestDRLearner(model_regression=model_ytb, model_propensity=model_tb, random_state=seed)
forestdrl.fit(Y1B_train, T_train, X=X_train, cache_values=True)
# Estimate treatment effects on test data
forestdrl_te = forestdrl.effect(X_test)
forestdrl_te_inference = forestdrl.effect_inference(X_test).population_summary(alpha=0.1, value=0, decimals=3, tol=0.001)
print(forestdrl_te_inference)

               Uncertainty of Mean Point Estimate              
mean_point stderr_mean zstat pvalue ci_mean_lower ci_mean_upper
---------------------------------------------------------------
     0.025       0.068 0.371   0.71        -0.086         0.137
      Distribution of Point Estimate     
std_point pct_point_lower pct_point_upper
-----------------------------------------
    0.024          -0.013           0.066
     Total Variance of Point Estimate     
stderr_point ci_point_lower ci_point_upper
------------------------------------------
       0.072         -0.088          0.144
------------------------------------------

Note: The stderr_mean is a conservative upper bound.


### 7.2 Calculate the robustness

In [ ]:
forestdrl.fit(Y=Y1B_test, T=T_test, X=X_test)
forestdrl_te_test = forestdrl.effect(X_test)

ERMSE = ((forestdrl_te-forestdrl_te_test)**2).mean()**(1/2)
print(ERMSE)

0.093820138601304


### 7.3 Calculate the RScore

In [ ]:
rscore = scorer_b.score(forestdrl)
print(rscore)

0.0610468198302937
